# Hyperspec

In [1]:
import hyperspy.api as hs                 # hyperspectral data processing and some useful image viewing
import h5py                               # to handle .h5 files
from matplotlib import pyplot as plt      # Plotting
import numpy as np                        # Maths
import cv2                                # Image analysis
from os import walk                       # To get filepaths automatically
from natsort import natsorted             # To easily resort file order
from datetime import datetime             # For easily parsing timestamps
import warnings
warnings.filterwarnings("ignore")         # Attempt to remove some unnecessary pyplot warnings
from scipy.optimize import curve_fit
import pandas as pd
from pathlib import Path
import optical_image_functions as oif

Set folder location of data.

In [14]:
#folder=r'C:\Users\es758\University of Cambridge\Alice Dearle - mini1 hyper-perov\example 2 hyperspectral\090321\\'
folder=r'C:\Users\Sauron\Documents\grl31\NanoDTC Mini - Edward\example 2 hyperspectral\090321\\'
fpaths = oif.get_filepaths(folder)

The hyperspectral microscope in MRC has software which outputs 2 files per time step, one being a rectified image stack which is what is wanted. The code below extracts the file paths for these rectified images alone.

In [15]:
rectified_fpaths = []
for fpath in fpaths:
    if fpath[-9:] == "(1)(1).h5":
        rectified_fpaths.append(fpath)

Inspect some of the properties of each image stack, and extract the pixel size.

In [22]:
fpath=rectified_fpaths[0]
print(fpath)
f = h5py.File(fpath,'r')
# Extract the image
print("Number of wavelengths per time step", len(f['Cube/Wavelength']))
pixel_size = oif.get_pixel_size(fpath)
print("Pixel size is", pixel_size, "µm/pixel")

C:\Users\Sauron\Documents\grl31\NanoDTC Mini - Edward\example 2 hyperspectral\090321\\Untitled(1)(1).h5
Number of wavelengths per time step 26
Pixel size is 0.325 µm/pixel


For further information of what each h5py file contains.

In [23]:
# oif.get_h5py_contents(fpath)

## Shape data into nice form

In [ ]:
def extract_h5py_image_stack(fpath):
    """
    Extracts hyperspectral data from h5y file and forms it into a hyperspy object.
    Data is returned as a hyperspy 2D signal (stack of images of different wavelengths).
    Data can be easily converted to 1D signal (wavelength spectrum at each pixel) by transposing the object.
    """
    f = h5py.File(fpath,'r')
    
    pixel_size = oif.get_pixel_size(fpath)
    
    wavelengths=np.array(f['Cube/Wavelength'])
    dwavelength=wavelengths[1]-wavelengths[0]
    print(dwavelength)
    images = np.array(f['Cube/Images']).astype("float32")
    
    lam_size, y_size, x_size = images.shape
    
    dictlam = {'size': lam_size, 'name':'lambda', 'units':'nm', 'scale':dwavelength, 'offset':wavelengths[0]}
    dicty = {'size': y_size, 'name':'y', 'units':'µm', 'scale':pixel_size, 'offset':0} # Note y still increases as go down the image
    dictx = {'size': x_size, 'name':'x', 'units':'µm', 'scale':pixel_size, 'offset':0}

    image_stack = hs.signals.Signal2D(images, axes =[dictlam, dicty, dictx])
    return image_stack

In [ ]:
%matplotlib qt


test_image_stack = extract_h5py_image_stack(fpath)
# test_image_stack.plot(navigator='auto')
# test_image_stack.axes_manager

im_1d = test_image_stack.transpose()
im_1d.plot(navigator='slider')
plt.show()

im_1d.plot()
plt.show()
im_1d.axes_manager

In [ ]:
# inspecting how the data is stored before stacking.

In [ ]:
%matplotlib inline
print(test_image_stack.data.shape)
print(test_image_stack.data[7][:][:].shape)
test=test_image_stack.data[24][:][:]
print(test.shape)
plt.imshow(test)

In [ ]:
%matplotlib inline
print(im_1d.data.shape)
print(im_1d.data[0].shape)
test=im_1d.data[:][:]
print(test[:,:,0].shape)
plt.imshow(test[:,:,24])

In [ ]:
svd = im_1d.decomposition()
im_1d.plot_explained_variance_ratio()
im_1d.plot_decomposition_loadings(comp_ids=4) # look at relevant sections of the image
im_1d.plot_decomposition_factors(comp_ids=4) # look at corresponding spectra
plt.show()

In [ ]:
#im_1d.decomposition(algorithm="sklearn_pca")
im_1d.decomposition(algorithm="NMF", output_dimension=4)
#im_1d.plot_explained_variance_ratio()

In [ ]:
%matplotlib inline
# set the number of components to whatever looks sensible from the scree plot
im_1d.plot_decomposition_loadings(comp_ids=4) # look at relevant sections of the image
plt.show()
im_1d.plot_decomposition_factors(comp_ids=4) # look at corresponding spectra
plt.show()

In [ ]:
# time points pick don't have to be that special, just different.
# same goes for space vs time plot.

In [5]:
x=np.array([1,9,3])
np.save("dumb_array", x)

In [6]:
y=np.load("dumb_array.npy")
y

array([1, 9, 3])

In [7]:
import numpy as np
from pathlib import Path

path = Path('~/data/tmp/').expanduser()
path.mkdir(parents=True, exist_ok=True)

lb,ub = -1,1
num_samples = 5
x = np.random.uniform(low=lb,high=ub,size=(1,num_samples))
y = x**2 + x + 2

np.save(path/'x', x)
np.save(path/'y', y)

x_loaded = np.load(path/'x.npy')
y_load = np.load(path/'y.npy')

print(x is x_loaded) # False
print(x == x_loaded) # [[ True  True  True  True  True]]

False
[[ True  True  True  True  True]]


In [8]:
path

WindowsPath('C:/Users/Edward/data/tmp')